In [ ]:
## Solve Every Sudoku Puzzle

## See http://norvig.com/sudoku.html

## Throughout this program we have:
##   r is a row,    e.g. 'A'
##   c is a column, e.g. '3'
##   s is a square, e.g. 'A3'
##   d is a digit,  e.g. '9'
##   u is a unit,   e.g. ['A1','B1','C1','D1','E1','F1','G1','H1','I1']
##   grid is a grid,e.g. 81 non-blank chars, e.g. starting with '.18...7...
##   values is a dict of possible values, e.g. {'A1':'12349', 'A2':'8', ...}

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u])
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

################ Unit Tests ################

def test():
    "A set of tests that must pass."
    assert len(squares) == 81
    assert len(unitlist) == 27
    assert all(len(units[s]) == 3 for s in squares)
    assert all(len(peers[s]) == 20 for s in squares)
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    print('All tests pass.')

################ Parse a Grid ################

def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    if len(chars) != 81: print(grid, chars, len(chars))
    assert len(chars) == 81
    return dict(zip(squares, chars))

################ Constraint Propagation ################

def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

################ Display as 2-D grid ################

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width) + ('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print()

################ Search ################

def solve(grid): return search(parse_grid(grid))

def search(values):
    "Using depth-first search and propagation, try all possible values."
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in squares):
        return values ## Solved!
    ## Chose the unfilled square s with the fewest possibilities
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    for d in values[s]:
        result = search(assign(values.copy(), s, d))
        if result: return result

################ System test ################

import time

def solve_all(grids, name=''):
    """Attempt to solve a sequence of grids. Report results."""
    times, results = zip(*[time_solve(grid) for grid in grids])
    N = len(results)
    if N > 0:
        print("Solved %d of %d %s puzzles (avg %.2f secs (%d Hz), max %.2f secs)." % (
            sum(results), N, name, sum(times)/N, N/sum(times), max(times)))
            
def time_solve(grid):
    start = time.clock()
    values = solve(grid)
    t = time.clock()-start
    print(values.values())
    return (t, solved(values))

def solved(values):
    "A puzzle is solved if each unit is a permutation of the digits 1 to 9."
    def unitsolved(unit): return set(values[s] for s in unit) == set(digits)
    return values is not False and all(unitsolved(unit) for unit in unitlist)


grid1  = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2  = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
hard1  = '.....6....59.....82....8....45........3........6..3.54...325..6..................'


In [ ]:
solve_all([grid2])

In [71]:
from random import randint, sample
from math import sqrt
import time

class SudokuSolver(object):
    def __init__(self, sudoku_size, init_values):
        self.sudoku_size = sudoku_size
        self.init_values = list(init_values)
        self.fixed_positions = []
        self.population = []
    def nodes(self):
        return self.sudoku_size * self.sudoku_size
    def kcolors(self):
        return self.sudoku_size
    def edges(self):
        return (self.nodes() * (self.kcolors()-1 + (self.kcolors()-(sqrt(self.kcolors()))) * 2)) / 2
    def population(self):
        return self.population
    def initPopulation(self, n_population):
        for i in range(0, n_population):
            solution = self.init_values[:]
            for i in range(0,len(solution)):
                c = solution[i]
                if c == 0 or c == '.':
                    solution[i] = randint(1,9)
                else:
                    solution[i] = int(solution[i])
                    self.fixed_positions.append(i)
            self.population.append(solution)
    def row_indexes(self, position):
        return range(int(position/self.sudoku_size)*9,int(position/self.sudoku_size)*9+9)
    def row_index(self, position):
        return int(position/self.sudoku_size)
    def col_index(self, position):
        return position%self.sudoku_size
    def col_indexes(self, position):
        return range(position%self.sudoku_size,73+(position%self.sudoku_size),9)
    def zone_number(self, position):
        row_index = self.row_index(position)
        col_index = self.col_index(position)
        if (row_index % 9 < 3):
            if (col_index % 9 < 3):
                return 0
            elif (col_index % 9 < 6):
                return 1
            else:
                return 2
        elif (row_index % 9 < 6):
            if (col_index % 9 < 3):
                return 3
            elif (col_index % 9 < 6):
                return 4
            else:
                return 5
        else:
            if (col_index % 9 < 3):
                return 6
            elif (col_index % 9 < 6):
                return 7
            else:
                return 8
    def zone_indexes(self, position):
        zone_number = self.zone_number(position)
        first = self.sudoku_size * int(zone_number/3)*3 + zone_number % 3 * 3
        indexes = []
        for i in range(0, 3):
            indexes.append(first) 
            indexes.append(1 + first)
            indexes.append(2 + first)
            first = first + self.sudoku_size
        return indexes
    def adyacents_indexes(self, position):
        indexes = set()
        [indexes.add(x) for x in self.col_indexes(position)]
        [indexes.add(x) for x in self.row_indexes(position)]
        [indexes.add(x) for x in self.zone_indexes(position)]
        indexes.remove(position)
        return indexes
    def fitness(self, solution):
        # Cantidad de ejes que unen nodos del mismo color
        bad_edges = 0
        for pos in range(0, self.nodes()):
            for ady in self.adyacents_indexes(pos):
                if (solution[ady] == solution[pos]):
                    bad_edges += 1
        return bad_edges / 2
    def mutation1(self, solution):
        # Colorear nodos con colores habilitados, exponencialmente
        keep_mutating = randint(1,10)
        mutations = 0
        while (keep_mutating > 2 and mutations < len(solution)):
            i = randint(0,len(solution)-1)
            if (not(i in self.fixed_positions)):
                available_colors = set(range(1,10))
                for ady in self.adyacents_indexes(i):
                    if (solution[ady] in available_colors):
                        available_colors.remove(solution[ady])
                if (available_colors):
                    solution[i] = sample(available_colors, 1)[0]
                else:
                    solution[i] = randint(1,9)          
            keep_mutating = randint(1,10)
            mutations += 1
        return solution
    def mutation2(self, solution):
        # Colorear nodos mal coloreados con colores random, exponencialmente
        keep_mutating = randint(1,10)
        mutations = 0
        while (keep_mutating > 2 and mutations < len(solution)):
            i = randint(0,len(solution)-1)
            if (not(i in self.fixed_positions)):
                bad = False
                for ady in self.adyacents_indexes(i):
                    if solution[ady] == solution[i]:
                        bad = True
                        break
                if (bad):
                    solution[i] = randint(1,9)          
            keep_mutating = randint(1,10)
            mutations += 1
        return solution
    def mutation3(self, solution):
        # Intercambiar filas
        row1 = randint(0,8)
        row2 = randint(0,8)
        tmp = solution[row1 * self.sudoku_size: row1 * self.sudoku_size + 9]
        solution[row1 * self.sudoku_size: row1 * self.sudoku_size + 9] = solution[row2 * self.sudoku_size: row2 * self.sudoku_size + 9]
        solution[row2 * self.sudoku_size: row2 * self.sudoku_size + 9] = tmp
        return solution
    def crossover(self, parent1, parent2):
        solution = []
        for i in range(0,81):
            if i % self.sudoku_size == 0:
                r = randint(0,1)
                if (r):
                    parent = parent1
                else:
                    parent = parent2
            solution.append(parent[i])
        return solution
    def solve(self, n_population):
        start_time = time.time()
        self.initPopulation(n_population)
        best_fitness = self.edges()
        iter_count = 0
        iter_without_improvement = 0
        while (best_fitness > 0 and iter_count < 1000 and iter_without_improvement < 150):
            self.population = sorted(self.population, key=self.fitness)
            parent1 = self.population[0]
            parent2 = self.population[1]
            new_best = self.fitness(parent1)
            if (new_best < best_fitness):
                best_fitness = new_best
                iter_without_improvement = 0
                print("iteration %d, new best_fitness %d" %(iter_count, best_fitness))
            else:
                iter_without_improvement += 1
            for i in range(int(n_population/2), n_population):
                child = self.crossover(parent1, parent2)
                mutation_choice = randint(1,3)
                if best_fitness < 20:
                    mutation_choice = (1,2)
                if (mutation_choice == 1):
                    self.population[i] = self.mutation1(child)
                elif (mutation_choice == 2):
                    self.population[i] = self.mutation2(child)
                else:
                    self.population[i] = self.mutation3(child)
            iter_count = iter_count + 1
        if (best_fitness == 0):
            print("--- Se resolvió el SUDOKU en %s segundos!!!! ---" % (time.time() - start_time))
        else:
            print("NO ANDUVO, se obtuvo un fitness de %d" % best_fitness)
        return self.population[0]

In [72]:
init_values_easy = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
init_values_hard = '.....6....59.....82....8....45........3........6..3.54...325..6..................'
s = SudokuSolver(9, init_values_easy)
print(s.solve(100))

iteration 0, new best_fitness 72
iteration 1, new best_fitness 63
iteration 2, new best_fitness 56
iteration 3, new best_fitness 47
iteration 4, new best_fitness 45
iteration 5, new best_fitness 43
iteration 6, new best_fitness 42
iteration 11, new best_fitness 40
iteration 14, new best_fitness 39
iteration 15, new best_fitness 37
iteration 20, new best_fitness 36
iteration 23, new best_fitness 35
iteration 24, new best_fitness 34
iteration 27, new best_fitness 33
iteration 30, new best_fitness 31
iteration 31, new best_fitness 30
iteration 33, new best_fitness 28
iteration 39, new best_fitness 27
iteration 43, new best_fitness 26
iteration 44, new best_fitness 25
iteration 51, new best_fitness 24
iteration 52, new best_fitness 23
iteration 53, new best_fitness 22
iteration 54, new best_fitness 21
iteration 55, new best_fitness 20
iteration 56, new best_fitness 19
NO ANDUVO, se obtuvo un fitness de 19
[4, 7, 1, 9, 3, 7, 8, 2, 5, 6, 3, 5, 4, 2, 8, 9, 1, 7, 4, 8, 2, 7, 6, 1, 3, 9, 5, 9, 

In [ ]:
['4', '1', '7', '3', '6', '9', '8', '2', '5', '6', '3', '2', '1', '5', '8', '9', '4', '7', '9', '5', '8', '7', '2', '4', '3', '1', '6', '8', '2', '5', '4', '3', '7', '1', '6', '9', '7', '9', '1', '5', '8', '6', '4', '3', '2', '3', '4', '6', '9', '1', '2', '7', '5', '8', '2', '8', '9', '6', '4', '3', '5', '7', '1', '5', '7', '3', '2', '9', '1', '6', '8', '4', '1', '6', '4', '8', '7', '5', '2', '9', '3']